In [ ]:
# Importing Dependencies
from lyricsgenius import Genius #uses BeautifulSoup, should think about use
import pprint as j
import api_key #importing api_key.py file that holds my personal client access token

In [ ]:
# Initiating Genius API
token = api_key.client_access_token
genius = Genius(token, remove_section_headers = True) 

# # Turn off status messages
# genius.verbose = False

# # Remove section headers (e.g. [Chorus]) from lyrics when searching
# genius.remove_section_headers = True

# # Include hits thought to be non-songs (e.g. track lists)
# genius.skip_non_songs = False

# # Exclude songs with these words in their title
# genius.excluded_terms = ["(Remix)", "(Live)"]

##### Searching for multiple songs by lyric query

In [ ]:
def search_song_by_lyric(lyric_query, return_lyrics=False):
    """
    Search for songs by a given lyric query and return the hits.

    Args:
    - lyric_query (str): The lyric query to search for.
    - return_lyrics (bool): Whether to include full lyrics in the results. Defaults to False. If True, takes a little longer

    Returns:
    - hits (list): Returns songs that match the query (sorted by Popularity) / List of dictionaries containing information about each hit (song).
    """
    request = genius.search_lyrics(lyric_query)
    hits = []
    for hit in request['sections'][0]['hits']:
        lyric_url = hit['result']['url']
        hit_info = {
            'genius_id': hit['result']['id'],
            'song': hit['result']['title'],
            'artists': hit['result']['artist_names'],
            'full_song_title': hit['result']['full_title'],
            'lyric_match': hit['highlights'][0]['value'],
            'full_lyrics_url': lyric_url,
        }
        if return_lyrics:
            hit_info['full_lyrics'] = genius.lyrics(song_url=lyric_url)
        hits.append(hit_info)
    return hits

In [ ]:
# Return all songs' lyrics (if return_lyrics = True)

# Lyric Query as str - (from: Can We Talk? by Tevin Campbell)
lyric_query = 'And just like a Roni' 

# searching for songs based on lyric query....saving them in variable 'hits'
hits = search_song_by_lyric(lyric_query, return_lyrics = False) #change to TRUE to return lyrics of all hits

# Show top hit song's info (most popular / likely)
hits[0] 

In [ ]:
# Alternate: Return just top song's (single song) lyrics 

lyric_query = 'And just like a Roni' #from: Can We Talk? by Tevin Campbell
hits = search_song_by_lyric(lyric_query, return_lyrics = False) #change to TRUE to return lyrics of all hits

#Getting Top Hit
top_songName = hits[0]['song']
top_songArtist = hits[0]['artists']

song_matched = genius.search_song(top_songName, top_songArtist)

# print(song_matched.artist)
# print(song_matched.title)
# print(song_matched.lyrics)

# Then can save lyrics: 
#song_matched.save_lyrics(filename=None, extension='json', overwrite=False, ensure_ascii=True, sanitize=True, verbose=True)

##### Getting Song Metadata (includes copyright, other credits / attribution, etc)
TODO: Clean up / dig into

In [ ]:
# Getting the ID of the top hit (single song)
top_hit_id = hits[0]['genius_id']

# Fetching full information about the top hit song // Ref: https://docs.genius.com/#songs-h2
top_hit_full_info = genius.song(song_id=top_hit_id)

# Getting basic song info
basic_song_info = top_hit_full_info['song']
# print(basic_song_info)  # Entire basic song info
# print(basic_song_info['title'])  # Title of the song
# print(basic_song_info['artist_names'])  # Names of the artists
# print(basic_song_info['release_date'])  # Release date of the song
# print(basic_song_info['recording_location'])  # Recording location of the song

# Accessing album information if available
album_info = basic_song_info['album']
print(album_info)  # Entire album info

# *** Getting other important song credits! Includes copyright / label info ****
song_credits = top_hit_full_info['song']['custom_performances']
# print(song_credits)  # Dictionary of song credits

# Accessing relevant people involved in the song
# print(top_hit_full_info['song']['featured_artists'])  # Featured artists
# print(top_hit_full_info['song']['primary_artist'])  # Primary artist
# print(top_hit_full_info['song']['producer_artists'])  # Producer artists
# print(top_hit_full_info['song']['writer_artists'])  # Writer artists

# Accessing song relationships such as samples, interpolations, etc.
# print(top_hit_full_info['song']['song_relationships'])  # Song relationships


In [ ]:
# Scrap: Digging into song credits / attribution
xtra_credits = top_hit_full_info['song']['custom_performances']
# j.pprint(xtra_credits)
# for item in xtra_credits:
#     if item['label'] == 'Copyright ©':
#         artists = item['artists']
#         for artist in artists:
#             print("Name:", artist['name'])
#             print("URL:", artist['url'])

for item in xtra_credits:
    label = item['label']
    artists = item['artists']
    print("Label:", label)
    for artist in artists:
        print("\tName:", artist['name'])
        print("\tID:", artist['id'])
        print("\tURL:", artist['url'])


In [ ]:
def extract_attribution_info(performance_list):
    performances_dict = {}

    for item in performance_list:
        label = item['label']
        artists = item['artists']
        artists_info = []

        for artist in artists:
            artist_info = (artist['name'], artist['id'], artist['url']) #dict of lists
            # artist_info = {'name': artist['name'], 'id': artist['id'], 'url': artist['url']} #dict of dicts
            artists_info.append(artist_info)

        performances_dict[label] = artists_info

    return performances_dict


xtra_credits = top_hit_full_info['song']['custom_performances']
performances_info = extract_attribution_info(xtra_credits)
j.pprint(performances_info)


In [ ]:
# Scrap Function / ignore

## Function to search song by lyric
lyric_query = 'And just like a Roni'

request = genius.search_lyrics(lyric_query) #only returns type=='lyric', equivalent to genius.search('[query]', type_='lyric')
#for each search result

# j.pprint(request)
for hit in request['sections'][0]['hits']: #request['sections']= 1
    # j.pprint(hit) #helpful to see everything the return gives

    # print(hit['result']['full_title']) #returns full title, can also return just song name by replacing ['full_title'] w/ ['title']
    print(hit['highlights'][0]['value']) #returns window where search term matches
#     print('----')
